## OHCL w/ Volume

In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np

from math import pi
from stochastic.continuous import FractionalBrownianMotion
from stochastic.noise import GaussianNoise
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column



def show_plot(t, hurst, freq='1h'):
    price_fbm = FractionalBrownianMotion(t=t, hurst=hurst)
    volume_gen = GaussianNoise(t=t*t)

    start_date = pd.to_datetime('2010-01-01', format='%Y-%m-%d')

    price_volatility = price_fbm.sample(t, zero=False)
    prices = price_volatility + 10000
    volume_volatility = volume_gen.sample(t) * np.random.lognormal(5, 1.5, t)
    volumes = volume_volatility * (10 * price_volatility) + 1

    price_frame = pd.DataFrame([], columns=['date', 'price'], dtype=float)
    volume_frame = pd.DataFrame(
        [], columns=['date', 'volume'], dtype=float)

    price_frame['date'] = pd.date_range(start=start_date, periods=t, freq="1min")
    price_frame['price'] = abs(prices)

    volume_frame['date'] = price_frame['date'].copy()
    volume_frame['volume'] = abs(volumes)

    price_frame.set_index('date')
    price_frame.index = pd.to_datetime(price_frame.index, unit='m', origin=start_date)

    volume_frame.set_index('date')
    volume_frame.index = pd.to_datetime(volume_frame.index, unit='m', origin=start_date)

    ohlc = price_frame['price'].resample(freq).ohlc()
    volumes = volume_frame['volume'].resample(freq).sum() / 1e7
    
    TOOLS = "xpan,wheel_zoom,box_zoom,crosshair,reset,save"
    
    price_plt = figure(title="Simulated Price",
                       y_axis_label='Price ($)',
                       x_axis_type='datetime',
                       tools=TOOLS)
    price_plt.xaxis.major_label_orientation = pi/4
    price_plt.grid.grid_line_alpha=0.3
    
    width = 60*1000*10
    inc = ohlc.close > ohlc.open
    dec = ohlc.open > ohlc.close
    
    price_plt.segment(ohlc.index, ohlc.high, ohlc.index, ohlc.low, color="black")
    price_plt.vbar(ohlc.index[inc], width, ohlc.open[inc], ohlc.close[inc], fill_color="#9BDE39", line_color="black")
    price_plt.vbar(ohlc.index[dec], width, ohlc.open[dec], ohlc.close[dec], fill_color="#F2583E", line_color="black")

    volume_plt = figure(title="Simulated Volume",
                        y_axis_label="Volume",
                        x_axis_label="Time",
                        x_axis_type="datetime",
                        x_range=price_plt.x_range,
                        plot_height=250,
                        tools=TOOLS)
    
    volume_plt.vbar(ohlc.index, bottom=0, top=volumes, width=width, color="#72B5C8", alpha=0.3)
    
    output_notebook()

    show(column(price_plt, volume_plt))

widgets.interact(show_plot,
                 t=widgets.IntSlider(min=1, max=1000000, value=100000),
                 hurst=widgets.FloatSlider(min=0.5, max=0.8, step=0.01, value=0.61));

interactive(children=(IntSlider(value=100000, description='t', max=1000000, min=1), FloatSlider(value=0.61, de…

## FBMExchange OHCLV Example

In [3]:
import sys
import os
import time
import warnings
import numpy as np

sys.path.append(os.path.dirname(os.path.abspath('')))

warnings.filterwarnings('ignore')

from tensortrade.exchanges.simulated import FBMExchange

exchange = FBMExchange()

exchange.reset()

TOOLS = "xpan,wheel_zoom,box_zoom,crosshair,reset,save"

price_plt = figure(title="Simulated Price",
                   y_axis_label='Price ($)',
                   x_axis_type='datetime',
                   tools=TOOLS)
price_plt.xaxis.major_label_orientation = pi/4
price_plt.grid.grid_line_alpha=0.3

width = 60*1000*10
inc = exchange.data_frame.close > exchange.data_frame.open
dec = exchange.data_frame.open > exchange.data_frame.close

price_plt.segment(exchange.data_frame.index, exchange.data_frame.high, exchange.data_frame.index, exchange.data_frame.low, color="black")
price_plt.vbar(exchange.data_frame.index[inc], width, exchange.data_frame.open[inc], exchange.data_frame.close[inc], fill_color="#9BDE39", line_color="black")
price_plt.vbar(exchange.data_frame.index[dec], width, exchange.data_frame.open[dec], exchange.data_frame.close[dec], fill_color="#F2583E", line_color="black")

volume_plt = figure(title="Simulated Volume",
                    y_axis_label="Volume",
                    x_axis_label="Time",
                    x_axis_type="datetime",
                    x_range=price_plt.x_range,
                    plot_height=250,
                    tools=TOOLS)

volume_plt.vbar(exchange.data_frame.index, bottom=0, top=exchange.data_frame['volume'], width=width, color="#72B5C8", alpha=0.3)

output_notebook()

show(column(price_plt, volume_plt))

W0822 09:32:51.117084 4528575936 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


Loading BokehJS ...

## Tick trade data and volume

In [2]:
# TODO